In [ ]:
import os
import sys
import requests

from dotenv import load_dotenv
import s3fs
from fastapi import FastAPI
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel
import uvicorn

PROJECT_PATH = '/home/onyxia/work/llm-open-data-insee/'
sys.path.append(os.path.join(PROJECT_PATH, 'src')) 

from config import RAG_PROMPT_TEMPLATE
from model_building import build_llm_model
from chain_building import load_retriever, build_chain

# Load env variables from .env file
load_dotenv()

# S3 configuration
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

## Query RAG directly

In [ ]:
from db_building import reload_database_from_local_dir
db = reload_database_from_local_dir(persist_directory="/home/onyxia/work/llm-open-data-insee/data/chroma_db")

In [ ]:
# Import Chroma DB from S3
from config import RAG_PROMPT_TEMPLATE, EMB_MODEL_NAME, MODEL_NAME

"""DB_PATH_S3 = os.path.join(os.environ["S3_BUCKET"], os.environ["DB_KEY_S3"])
DB_PATH_LOCAL = os.path.join(PROJECT_PATH, "data", "chroma_db")
print("database : " , DB_PATH_LOCAL)
fs.get(DB_PATH_S3, DB_PATH_LOCAL, recursive=True)
"""
# Generate prompt template
prompt = PromptTemplate(input_variables=["context", "question"], template=RAG_PROMPT_TEMPLATE)
# Create a pipeline with tokenizer and LLM
retriever = load_retriever(emb_model_name=EMB_MODEL_NAME, persist_directory = DB_PATH_LOCAL)
llm = build_llm_model(model_name=MODEL_NAME,
                    quantization_config=True, 
                    config=True, 
                    token=os.environ["HF_TOKEN"])
chain = build_chain(retriever, prompt, llm)

In [ ]:
question = "Comment la France a-t-elle gérée le COVID? " 
response = chain.invoke(question)
response

In [ ]:
print("Question : " , response["question"])
print("Answer : ", response["answer"])
print("Documents : ", " ".join([f"\nDoc {i} : ({doc.metadata["title"]},{doc.metadata["source"]}) {doc.page_content}" for i, doc in enumerate(response["context"])])) 

In [ ]:
def extract_context_as_dict(response):
    dict_context = {}
    for i, doc in enumerate(response['context']):
        dict_context[i] = doc.metadata
        dict_context[i]['content'] = doc.page_content
    return dict_context

## Query RAG through locally deployed API

In [ ]:
CHATBOT_URL = "https://user-avouacr-879366-user.user.lab.sspcloud.fr/chat"

In [ ]:
question = "Je cherche à connaitre le nombre (et eventuellement les caractéristiques) des véhicules 'primes à la conversion' dans plusieurs départements d'occitanie, en particulier l'aveyron."
query = {"question": question}
output = requests.post(CHATBOT_URL, json=query)
print(output)

In [ ]:
output.json()